# Web Accessibility
---

## Video: ¿Qué es la accesibilidad web?

[¿Qué es la accesibilidad web?](https://www.youtube.com/watch?v=gQIKQO41pME)

## a11y

![a11y](img/aw_01_intro_a11y.png)

## Problemas de accesibilidad en el mundo real

![Problemas de accesibilidad en el mundo real](img/aw_01_intro_problemas.png)

## Definición AW

![Definición de accesibilidad web](img/aw_taller_definicion.png)

## Ejemplo

### Lo que estamos viendo

![Lo que estamos viendo](img/aw_03_discapacidad_ejemplo_ven.png)

### Lo que ven las personas ciegas

![Lo que ven las personas ciegas](img/aw_03_discapacidad_ejemplo_noven.png)

### Datos estadísticos

![Discapacidad en el mundo](img/aw_03_discapacidad_estadistica.png)

## Legislación

### Pautas y Leyes a nivel mundial

![W3C](img/aw_04_pautas_y_leyes_w3c.png)

|              |                                                                               |
| --           | --                                                                            |
| **W3C**      | World Wide Web Consortium                                                     |
| **WAI**      | Web Accessibility Initiative                                                  |
| **WCAG 2.0** | Web Content Accessibility Guidelines 2.0 [URL](https://www.w3.org/TR/WCAG20/) |

### WCAG 2.0

![WCAG 2.0](img/aw_04_pautas_y_leyes_wcag2.0_español.png)

### Técnicas

| Criterio 1.1.1 |
| --- |
| ![CAPTCHA](img/aw_05_tecnicas_captcha.png) |

| Criterio 1.2.2 |
| --- |
| ![Video CC](img/aw_05_tecnicas_videoscc.jpeg) |

| Criterio 1.1.1 |
| --- |
| ![Imágenes con atributo ALT](img/aw_05_tecnicas_alt.jpg) |

| Criterio 1.4.2 |
| --- |
| ![Pausar audio](img/aw_05_tecnicas_audio.jpeg) |

| Criterio 1.4.3 |
| --- |
| ![Alto contraste](img/aw_05_tecnicas_contraste.png) |
| ![Ejemplo Alto contraste](img/aw_05_tecnicas_contraste2.png) |
| Ejemplo: Poco contraste vs Mucho contraste |

| Criterio 2.4.2 |
| --- |
| ![Titulos y encabezados](img/aw_05_tecnicas_encabezados.jpg) |

| Criterio 2.4.4 |
| --- |
| ![Textos descriptivos](img/aw_05_tecnicas_texto_descriptivo.png) |

| Criterio 2.1.1 |
| --- |
| ![Mouse + Teclado](img/aw_05_tecnicas_mouse_teclado.jpg) |

| Criterio 2.4.3 |
| --- |
| ![Foco](img/aw_05_tecnicas_foco.jpg) |

| Criterio 2.2.1 |
| --- |
| ![Tiempo suficiente](img/aw_05_tecnicas_tiempo.png) |

| Criterio 2.3.1 |
| --- |
| ![Destellos](img/aw_05_tecnicas_destellos.png) |

| |
| -- |
| ![Palabras simples y párrafos breves](img/aw_05_tecnicas_palabras_simples.png) |

| |
| -- |
| ![Diseño responsivo](img/aw_05_tecnicas_disenio_responsivo.png) |

| Criterio 3.3.1 |
| --- |
| ![Diseño responsivo](img/aw_05_tecnicas_disenio_responsivo2.png) |

### Video: ¿Cómo diseñar un sitio web accesible?

[¿Cómo diseñar un sitio web accesible?](https://www.youtube.com/watch?v=5WrB508ZpNI)

### Legislación en Argentina

![Legislación en Argentina](img/aw_04_pautas_y_leyes_arg.png)
[ONTI](https://www.argentina.gob.ar/jefatura/innovacion-publica/onti/evaluacion-accesibilidad-web)